In [16]:
%load_ext autoreload
%autoreload 2
import numpy as np
import os
import numpy as np
import pandas as pd
import sys
import matplotlib.pyplot as plt
import argparse
import pickle 
from src.utils import ReaderWriter, create_directory

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
cmd_opt = argparse.ArgumentParser(description='Argparser for data')
cmd_opt.add_argument('-data_dir',  type=str, default = './data/', help = 'directory of the data')
cmd_opt.add_argument('-target_dir',  type=str, default='processed',  help = 'folder name to save the processed data')
#cmd_opt.add_argument('-working_dir',  type=str, help = 'the main working directory')
cmd_opt.add_argument('-data_name',  type=str, help = '')
cmd_opt.add_argument('-feature_list',  type=str, help = 'list of feature names we are gonna consider')
cmd_opt.add_argument('-used_for',type=str, help = '') 
args, _ = cmd_opt.parse_known_args()

In [18]:
#args.data_dir = './data/'
args.data_name = 'TnpB_nuclease_screen_for_ML_with_features.csv'
#args.target_dir = 'processed'

In [19]:
##1. read the data
df = pd.read_csv(os.path.join(args.data_dir,args.data_name ),
                 header=0, 
                 delimiter=',' )
df

,Unnamed: 0,Identifier,spacer_20bp,average_editing_percent,used_for,polytvalues,polygvalues,polyavalues,polycvalues,Proto_GC_content,...,TTGC,TTGT,TTCA,TTCG,TTCC,TTCT,TTTA,TTTG,TTTC,TTTT
0,0,Seq_0,AGCAACCTGCCCAGGGCCTC,8.30,train,1,3,2,3,70,...,0,0,0,0,0,0,0,0,0,0
1,1,Seq_1,CACCTGATCACAGCGCTCCT,9.29,train,1,1,1,2,60,...,0,0,0,0,0,0,0,0,0,0
2,2,Seq_2,CTTCGGTACAGAGTGTTGGA,0.00,train,2,2,1,1,50,...,0,0,0,1,0,0,0,0,0,0
3,3,Seq_3,ACTACTTTTGATCCGGCCCA,1.81,train,4,2,1,3,50,...,0,0,0,0,0,0,0,1,0,1
4,4,Seq_4,AAAACCACACCTGTAGTAAG,36.38,train,1,1,4,2,40,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10206,10206,Seq_10206,CCTCTTAGACCTCTTTAAAT,0.00,train,3,1,3,2,35,...,0,0,0,0,0,0,1,0,0,0
10207,10207,Seq_10207,GCAAATTTGTGATTCTAAGT,1.54,train,3,1,3,1,30,...,0,1,0,0,0,1,0,1,0,0
10208,10208,Seq_10208,CCCAGCACTCGGAGGAGGCA,1.22,train,1,2,1,3,70,...,0,0,0,0,0,0,0,0,0,0
10209,10209,Seq_10209,CCCTCAGGGTCCCCTGGATC,1.99,train,1,3,1,4,70,...,0,0,0,0,0,0,0,0,0,0


In [20]:
args.feature_list = ['polytvalues', 'polygvalues', 'polyavalues', 'polycvalues',
       'Proto_GC_content', 'Proto_GC_count', 'spacermt', 'MFE_scaffold_spacer_HDV', 'MFE_spacer_HDV',
       'MFE_scaffold_spacer']

In [21]:
## remove the samples assigned for test
df_left = df[df['used_for']=='train']
df_main = df_left[['spacer_20bp', 'average_editing_percent']]
extended_featurs = df.columns[5:12].tolist() + df.columns[15:].tolist()
Non_protospacer_features = df_left[args.feature_list]
df_extended_features = df_left[extended_featurs]


In [22]:
from src.utils import get_char, outer_cross_val
x_extended_f = np.array(df_extended_features)
x_non_protos_f =  np.array(Non_protospacer_features)
y = np.array(df_main['average_editing_percent'])
## protospacer
protospacer = df_main['spacer_20bp'].apply(get_char)
num_nucl = len(protospacer.columns)
protospacer.columns = [f'B_encoded{i}' for  i in range(1, num_nucl+1)]
protospacer.replace(['A', 'C', 'T', 'G'], [0,1,2,3], inplace=True)
x_protospacer = np.array(protospacer)

In [13]:
data_partitions = outer_cross_val(x_protospacer.shape[0], 5)


[   0    1    2 ... 2035 2036 2037]
run_num: 0
train data:8150/10188 = 0.7999607381232823
test data:2038/10188 = 0.2000392618767177
[2038 2039 2040 ... 4073 4074 4075]
run_num: 1
train data:8150/10188 = 0.7999607381232823
test data:2038/10188 = 0.2000392618767177
[4076 4077 4078 ... 6111 6112 6113]
run_num: 2
train data:8150/10188 = 0.7999607381232823
test data:2038/10188 = 0.2000392618767177
[6114 6115 6116 ... 8148 8149 8150]
run_num: 3
train data:8151/10188 = 0.8000588928150766
test data:2037/10188 = 0.19994110718492344
[ 8151  8152  8153 ... 10185 10186 10187]
run_num: 4
train data:8151/10188 = 0.8000588928150766
test data:2037/10188 = 0.19994110718492344


In [23]:
## create directory to save the processed data
dir_processed = create_directory(args.target_dir, args.data_dir)
processed_data = [x_protospacer, x_extended_f,x_non_protos_f ,y]

In [24]:
ReaderWriter.dump_data(data_partitions, os.path.join(dir_processed, f'data_partitions.pkl'))
ReaderWriter.dump_data(processed_data, os.path.join(dir_processed, 'list_of_x_f_y.pkl'))


In [35]:
data_partitions

{0: {'train_index': array([ 2038,  2039,  2040, ..., 10185, 10186, 10187]),
  'test_index': array([   0,    1,    2, ..., 2035, 2036, 2037])},
 1: {'train_index': array([    0,     1,     2, ..., 10185, 10186, 10187]),
  'test_index': array([2038, 2039, 2040, ..., 4073, 4074, 4075])},
 2: {'train_index': array([    0,     1,     2, ..., 10185, 10186, 10187]),
  'test_index': array([4076, 4077, 4078, ..., 6111, 6112, 6113])},
 3: {'train_index': array([    0,     1,     2, ..., 10185, 10186, 10187]),
  'test_index': array([6114, 6115, 6116, ..., 8148, 8149, 8150])},
 4: {'train_index': array([   0,    1,    2, ..., 8148, 8149, 8150]),
  'test_index': array([ 8151,  8152,  8153, ..., 10185, 10186, 10187])}}

In [37]:
data_partitions[0]

{'train_index': array([ 2038,  2039,  2040, ..., 10185, 10186, 10187]),
 'test_index': array([   0,    1,    2, ..., 2035, 2036, 2037])}